# Convert given array as a sparse connected fcn

- Input node cannot be connected to the others
- But all output nodes should be connected to at least one hidden node or input node

In [1]:
import numpy as np
import torch
import torch.nn as nn

## Connection also represents its acitvation function
- 0 : not connected
- 1 : linear 
- 2 : ReLU
- 3 : Sigmoid
- and so on..

**this index can be changed**

In [29]:
# 0 : Not connected
# 1 : linear
# 2 : ReLU
# 3 : Sigmoid
activations = [None, None, nn.ReLU(), nn.Sigmoid()]

# References
- [Concatenate layer output with additional input data](https://discuss.pytorch.org/t/concatenate-layer-output-with-additional-input-data/20462)

In [2]:
class MatrixForWANN() : 
    def __init__(self, mat, in_dim, out_dim) : 
        
        # get when intiliazed
        self.mat = mat
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        # calculate
        self.num_hidden_nodes = self.mat.shape[1]-self.in_dim
        
        # when matrix has hidden layer
        if mat.shape[1] != in_dim : 
            self.hidden_dim = self.get_hidden_dim()
        else :
            self.hidden_dim = []
        
    def get_hidden_dim(self):
        in_dim = self.in_dim
        out_dim = self.out_dim
        mat_mask = self.mat
        hidden_dim_list = []
        start_col_idx = 0 
        finish_col_idx = in_dim-1

        while(True):

            if finish_col_idx >= mat_mask.shape[1]:
                break

            for i in range(start_col_idx, len(mat_mask)): 
                if (mat_mask[i,start_col_idx:(finish_col_idx + 1)].sum() == 0):
                    hidden_dim = i - sum(hidden_dim_list)
                    hidden_dim_list += [hidden_dim] 
                    start_col_idx = finish_col_idx + 1
                    finish_col_idx += i 
                    break
        return hidden_dim_list

# Examples

## Example 1

![](img/example1_1.png)

![](img/example1_2.png)

In [3]:
mat1 = np.array([[0,2,0,0,2,0,0,0,0,0],
                [2,0,2,0,0,0,0,0,0,0],
                [0,2,0,2,0,0,0,0,0,0],
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,0,0,0,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,3],
                [0,0,0,0,0,0,0,0,3,0]])
in_dim = 5
out_dim = 2
mat_wann1 = MatrixForWANN(mat1, in_dim, out_dim)

In [21]:
# first position represents row : FROM
mat1[0]

array([0, 2, 0, 0, 2, 0, 0, 0, 0, 0])

In [22]:
# second position represents column : TO
mat1[:, 0]

array([0, 2, 0, 0, 0, 0, 0])

In [23]:
# get destinations of input layer
mat1[:mat_wann1.num_hidden_nodes, :in_dim]

array([[0, 2, 0],
       [2, 0, 2],
       [0, 2, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [24]:
# num of total hidden nodes
mat_wann1.num_hidden_nodes

5

In [25]:
# get number of hidden layers and its node index
mat_wann1.hidden_dim

[3, 2]

## Example 2

![](img/example2_1.png)

![](img/example2_2.png)

In [5]:
mat2 = np.array([[2,0,0,0,0,0],
                [0,0,0,3,0,1]])
in_dim = 5
out_dim = 1
mat_wann2 = MatrixForWANN(mat2, in_dim, out_dim)

KeyboardInterrupt: 

In [26]:
mat_wann2.num_hidden_nodes

NameError: name 'mat_wann2' is not defined

In [ ]:
mat_wann2.hidden_dim

# Example 3

![](img/example3_1.png)

In [6]:
mat3 = np.array([[2,0,0,3,0],
                [0,0,0,0,1]])
in_dim = 5
out_dim = 2
mat_wann3 = MatrixForWANN(mat3, in_dim, out_dim)

In [27]:
mat_wann3.num_hidden_nodes

0

In [7]:
mat_wann3.hidden_dim

[]

## Example 4

![](img/example4_1.png)

In [33]:
mat4 = np.array([[3,0,1,0,0],
                [0,3,0,0,0],
                [0,0,0,2,0],
                [0,0,0,0,2],
                [0,0,1,2,0],
                [0,0,1,0,0]])
in_dim = 3
out_dim = 4
mat_wann4 = MatrixForWANN(mat4, in_dim, out_dim)

In [28]:
mat_wann4.num_hidden_nodes

2

In [9]:
mat_wann4.hidden_dim

[2, 3]

## Example 5
- Example 1에서 Input4 -> Hidden 5로 가는 Skip connection이 추가됨

![](img/example5_1.png)

In [32]:
mat5 = np.array([[0,2,0,0,2,0,0,0,0,0],
                [2,0,2,0,0,0,0,0,0,0],
                [0,2,0,2,0,0,0,0,0,0],
                [0,0,0,0,0,1,1,0,0,0],
                [0,0,0,2,0,0,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,3],
                [0,0,0,0,0,0,0,0,3,0]])
in_dim = 5
out_dim = 2
mat_wann5 = MatrixForWANN(mat5, in_dim, out_dim)

# class WANNFCN

In [31]:
def wrap_activation(x, idx_activation, activations) :  
    if idx_activation == 0 :
        assert True
    elif idx_activation == 1 :
        return nn.Linear(1,1)(x)
    else : 
        return activations[idx_activation](nn.Linear(1,1)(x))

In [ ]:
class WANNFCN(nn.Module) : 
    def __init__(self, mat_wann, activations) : 
        super(WANNFCN, self).__init__()
        self.mat = mat_wann.mat
        self.in_dim = mat_wann.in_dim
        self.out_dim = mat_wann.out_dim
        self.num_hidden_nodes = mat_wann.num_hidden_nodes
        self.hidden_dim = mat_wann.hidden_dim
        
        self.activations = activations
        
    def forward(self, x) : 
        return
    
    def to_hidden(self, x_input) :
        # input layer와 모든 이전 hidden layer를 참고
        # 그렇지 않으면 skip connection을 놓칠수 있음
        return
    
    def to_output(self, x) : 
        # input layer와 모든 이전 hidden layer를 탐색
        # 그렇지 않으면 skip connection을 놓칠수 있음
        return 